In [27]:
# !pip install py-readability-metrics
# !python -m nltk.downloader punkt

In [28]:
from langsmith import Client 

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

Inputs are provided to your model, so it know what to generate

In [30]:
client = Client()
dataset_name = "Rap Battle Dataset"

In [ ]:
# Inputs are provided to your model, so it know what to generate
dataset_inputs = [
    How to get ready for school
How to identify shapes
How to read body language
How to understand facial expressions
How to share with others
How to empathise with others
How to handle identify emotions
How to deal with emotions
How to spell their name
How to solve a maze
How to tidy up
How to ask for help
How to brush teeth
How to use the toilet
How to deal with loud noises
How to deal with bright lights
How to deal with strong smells
How to make friends
How to help others
  # ... add more as desired
]

# Outputs are provided to the evaluator, so it knows what to compare to
# Outputs are optional but recommended.
dataset_outputs = [
    {"must_mention": ["lawyer", "justice"]},
    {"must_mention": ["plastic", "nuclear"]},
]

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Rap battle prompts.",
)

client.create_examples(
    inputs=[{"question": q} for q in dataset_inputs],
    outputs=dataset_outputs,
    dataset_id=dataset.id,
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain import HuggingFaceHub

llm = HuggingFaceHub(repo_id="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO")
eval_llm = HuggingFaceHub(repo_id="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO")
prompt = ChatPromptTemplate.from_messages([("human", "Spit some bars about {question}.")])
chain = prompt | llm | StrOutputParser()

In [36]:
from langchain.smith import RunEvalConfig, run_on_dataset
from langsmith.evaluation import EvaluationResult, run_evaluator
from readability import Readability
 
@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

@run_evaluator
def Readability_eval(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    try:
        score = Readability(prediction).spache().score
    except:
        score = -1
    return EvaluationResult(key="Readability", score=score)
    
eval_config = RunEvalConfig(
    eval_llm=llm,
    custom_evaluators=[must_mention, Readability_eval],
    # You can also use a prebuilt evaluator
    # by providing a name or RunEvalConfig.<configured evaluator>
    evaluators=[
        # You can specify an evaluator by name/enum.
        # In this case, the default criterion is "helpfulness"
        "criteria",
        # Or you can configure the evaluator
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria(
            {
                "cliche": "Are the lyrics cliche?"
                " Respond Y if they are, N if they're entirely unique."
            }
        ),
    ],
)

In [38]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=chain,
    evaluation=eval_config,
    verbose=True,
    project_name="runnable-test-6",
    # Any experiment metadata can be specified here
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'runnable-test-6' at:
https://smith.langchain.com/o/cb290b0a-415d-5ed7-8ecf-fc36743479bf/datasets/b7760b0b-4480-41b3-b6a4-bd613fdb7343/compare?selectedSessions=7e693132-0841-4581-9be4-49ee40ea48db

View all tests for Dataset Rap Battle Dataset at:
https://smith.langchain.com/o/cb290b0a-415d-5ed7-8ecf-fc36743479bf/datasets/b7760b0b-4480-41b3-b6a4-bd613fdb7343
[------------------------------------------------->] 2/2

,output,feedback.helpfulness,feedback.harmfulness,feedback.cliche,feedback.must_mention,feedback.Readability,error,execution_time,run_id
count,2,0,0,0,2,2.0,0,2.000000,2
unique,2,0,0,0,1,NaN,0,NaN,2
top,"\nMegabyte: John Oppenheimer? Wait, the Nazi? ...",NaN,NaN,NaN,False,NaN,NaN,NaN,30a49ae4-caa4-46ca-bb78-6b0acbc693d5
freq,1,NaN,NaN,NaN,2,NaN,NaN,NaN,1
mean,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,2.116944,NaN
std,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.056582,NaN
min,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,2.076934,NaN
25%,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,2.096939,NaN
50%,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,2.116944,NaN
75%,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,2.136948,NaN


{'project_name': 'runnable-test-6',
 'results': {'11337c2f-f701-4a97-9572-1b80bbf97dda': {'input': {'question': 'a rap battle between Barbie and Oppenheimer'},
   'feedback': [EvaluationResult(key='helpfulness', score=None, value='[Step 2]: Analyze the submission - The submission is cleverly written: using the concept of "gammas" (a type of radiation) and', comment='[BEGIN REASONING]\n***\n[Step 1]: Analyze the input - The input is a suggestion for a rap battle between Barbie and Oppenheimer. The battle has to come from Oppenheimer, who is a nuclear physicist that invented the atomic bomb.\n\n[Step 2]: Analyze the submission - The submission is cleverly written: using the concept of "gammas" (a type of radiation) and', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('acadebc5-958a-40c2-b9d3-145cb4a598c4'))}, source_run_id=None, target_run_id=None),
    EvaluationResult(key='harmfulness', score=None, value='step 4: Confirm by analyzing if the phrases', comment="[add any pe